In [3]:
import nltk
from nltk.tokenize import word_tokenize

# Download the tokenizer if you haven't already
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
from joblib import load

word_to_index = load('Embeddings/word_to_index.joblib')

In [5]:
def loadModel():
    import numpy as np
    import tensorflow as tf
    np.random.seed(0)
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
    from tensorflow.keras.layers import Embedding
    from tensorflow.keras.preprocessing import sequence
    from tensorflow.keras.initializers import glorot_uniform
    np.random.seed(1)
    
    # Registering the Custom Embedding Layer
    @tf.keras.utils.register_keras_serializable(package="CustomLayers")
    class SerializableEmbedding(Embedding):
        def get_config(self):
            config = super().get_config()
            return config

    model=tf.keras.models.load_model('Review Detector2.keras')

    return model

In [6]:
def convert_and_pad(tokens, max_len=510, pad_value=0, unk_index=0):
    # Convert words to indices using the dictionary, use unk_index if word not found
    indexed_tokens=[]
    for word in tokens:
        if word in word_to_index.keys():
            idx=word_to_index.get(word)
            indexed_tokens.append(idx)
    #indexed_tokens = [word_to_index.get(word, unk_index) for word in tokens]
    
    # Pad or truncate to the required length
    if len(indexed_tokens) < max_len:
        indexed_tokens += [pad_value] * (max_len - len(indexed_tokens))  # Pad
    else:
        indexed_tokens = indexed_tokens[:max_len]  # Truncate if too long
    
    return indexed_tokens

In [28]:
def predict(text):
    # Example sentence
    sentence = text.lower()
    
    # Tokenizing the sentence
    tokens = word_tokenize(sentence)

    indexed_tokens=convert_and_pad(tokens)

    data=np.array(indexed_tokens).reshape(1,-1)

    prediction=model.predict(data)
    prediction=prediction[0][0]
    if(prediction>0.5):
        return ["Human Written",round(prediction*100,2)]
    else:
        return ["Machine Generated",round((1-prediction)*100,2)]
    

In [10]:
import numpy as np

In [7]:
model=loadModel()

2025-02-17 12:34:37.275564: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-17 12:34:37.447311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-17 12:34:37.532603: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-17 12:34:37.533388: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 12:34:37.676344: I tensorflow/core/platform/cpu_feature_gua

In [31]:
review="I enjoy watching this movie.  I love the movie.  It's one of the funniest"
pred=predict(review)
print("Review Analysis:",pred[0],f' ({pred[1]}%)')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Review Analysis: Machine Generated  (99.98%)
